In [ ]:
jimport pandas as pd
!pip install openai
import json
!pip install tiktoken
import tiktoken
import re


In [ ]:
#split="train"
df= pd.read_csv('2019_2022_DMVcollisionREPORT.csv')


#df = df.head(20)
original_num_cols= df.shape[1]



In [ ]:
import pandas as pd





# Specify the columns to be processed
columns_to_process = df.columns[12:df.shape[1]]

# Specify the columns to be left as is
columns_to_leave_as_is = ['AccTYPE(Mode)','AVnoOFVehicles', 'V2year', 'V2model','V2hitOTHERSspecify', 'V2noOFVehicles', 'AccDETAILS' ]

# Apply the function to generate feature labels for specified columns
for column in columns_to_process:
    if column not in columns_to_leave_as_is:
        df[column + '-Status'] = [f"{column}-{ 'Yes' if value == 1}" for value in df[column]]

# Print the DataFrame with new columns
print(df)


           AV_manufacturer  AVyear      AVmake     AVmodel    AVregloc  \
0           Aimotive , Inc  2010.0      Toyota       Prius          CA   
1                Apple Inc  2017.0       Lexus     RX 450h  California   
2    Aurora Innovation Inc  2017.0     Lincoln         MKZ  California   
3               Cruise LLC  2020.0  Cheverolet        Bolt          CA   
4               Cruise LLC  2020.0   Chevrolet        Bolt          CA   
..                     ...     ...         ...         ...         ...   
322               Zoox Inc  2016.0      Toyota  Highlander          CA   
323               Zoox Inc  2016.0      Toyota  Highlander          CA   
324               Zoox Inc  2016.0      Toyota  Highlander          CA   
325               Zoox Inc  2016.0      Toyota  Highlander          CA   
326               Zoox Inc  2016.0      Toyota  Highlander          CA   

     AccidentZIPcode AccidentSTATE AccidentCOUNTY AccidentCITY  \
0            94043.0            CA    Santa C

<ipython-input-15-a0cf760aec03>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '-Status'] = [f"{column}-{ 'Yes' if value == 1 else 'No'}" for value in df[column]]
<ipython-input-15-a0cf760aec03>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '-Status'] = [f"{column}-{ 'Yes' if value == 1 else 'No'}" for value in df[column]]
<ipython-input-15-a0cf760aec03>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

In [ ]:
df

,AV_manufacturer,AVyear,AVmake,AVmodel,AVregloc,AccidentZIPcode,AccidentSTATE,AccidentCOUNTY,AccidentCITY,AccidentADDRESS,...,AccOTHERfactorsSTOPGOTRAFFIC-Status,AccOTHERfactorsENTERINGORLEAVINGRAMP-Status,AccOTHERfactorsPREVIOUSCOLLISION-Status,AccOTHERfactorsUNFAMILIARWITHROAD-Status,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-Status,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-Status,AccOTHERfactorsUNINVOLVEDVEHICLE-Status,AccOTHERfactorsOTHER-Status,AccOTHERfactorsNONEAPPARENT-Status,AccOTHERfactorsRUNAWAYVEHICLE-Status
0,"Aimotive , Inc",2010.0,Toyota,Prius,CA,94043.0,CA,Santa Clara,View,"SB , 101 US Freeway Old Middlefield Rd x Shorline",...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
1,Apple Inc,2017.0,Lexus,RX 450h,California,94087.0,CA,Santa Clara,NaN,Sunnyvale Community Center Park Entrance,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
2,Aurora Innovation Inc,2017.0,Lincoln,MKZ,California,94107.0,CA,San Francisco,Francisco,Townsend 3rd,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-Yes,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
3,Cruise LLC,2020.0,Cheverolet,Bolt,CA,94109.0,CA,San Francisco,Francisco,Hyde St Sutter,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
4,Cruise LLC,2020.0,Chevrolet,Bolt,CA,94111.0,CA,San Francisco,Francisco,The Embarcadero Battery St,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Zoox Inc,2016.0,Toyota,Highlander,CA,94105.0,CA,San Francisco,Francisco,Folsom St near Essex,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
323,Zoox Inc,2016.0,Toyota,Highlander,CA,94103.0,CA,San Francisco,Francisco,10th Street Folsom Intersection,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
324,Zoox Inc,2016.0,Toyota,Highlander,CA,94133.0,CA,San Francisco,Francisco,"Pacific Ave , Kearny St Columbus Intersectio",...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVI

In [ ]:
columns_to_process = df.columns[12:original_num_cols]

# Specify the columns to be left as is
columns_to_leave_as_is = ['AccTYPE(Mode)','AVnoOFVehicles', 'V2year', 'V2model','V2hitOTHERSspecify', 'V2noOFVehicles', 'AccDETAILS' ]

elements_not_in_B = [element for element in columns_to_process if element not in columns_to_leave_as_is]

print(elements_not_in_B)
#df_processed = df.drop(columns=[col for col in columns_to_process if col not in columns_to_leave_as_is])
#print(df_processed)

['AccidenttimeOFdayAM', 'AccidenttimeOFdayPM', 'AVmotionMOVING', 'AVmotionSTOPPED', 'AVhitPEDESTRAIN', 'AVhitBICYCLIST', 'AVhitOTHERS', 'AVhitOTHERSspecify', 'AVdamagelevelUNKNOWN', 'AVdamagelevelNONE', 'AVdamagelevelMINOR', 'AVdamagelevelMODERATE', 'AVdamagelevelMAJOR', 'AVdamagePartLRC1', 'AVdamagePartRB', 'AVdamagePartRRC1', 'AVdamagePartLRPS1', 'AVdamagePartLRPS2', 'AVdamagePartRRPS1', 'AVdamagePartRRPS2', 'AVdamagePartLRPS3', 'AVdamagePartLRPS4', 'AVdamagePartRRPS3', 'AVdamagePartRRPS4', 'AVdamagePartFDS1', 'AVdamagePartFDS2', 'AVdamagePartFPS1', 'AVdamagePartFPS2', 'AVdamagePartFDS3', 'AVdamagePartFDS4', 'AVdamagePartFPS3', 'AVdamagePartFPS4', 'AVdamagePartLFC1', 'AVdamagePartFC1', 'AVdamagePartRC1', 'AVdamagePartFRC1', 'AVdamagePartLFC2', 'AVdamagePartFB', 'AVdamagePartFRC2', 'AVdamagePartLRC2', 'AVdamagePartLRC3', 'AVdamagePartRRC2', 'AVdamagePartRRC3', 'V2motionMOVING', 'V2motionSTOPPED', 'V2hitPEDESTRAIN', 'V2hitBICYCLIST', 'V2hitOTHERS', 'P1INJURED', 'P1PROPERTY', 'P1BICYCLI

In [ ]:
df_processed = df.drop(elements_not_in_B, axis=1)
#accDetails_df = df['AccDETAILS']
#df_processed = df_processed.drop('AccDETAILS', axis=1)

#df_processed['AccDETAILS']

In [ ]:
df_processed['AccDETAILS']

0      On 09/16/2019 at approximately 9:55 AM, an Aim...
1      On September 19th, 2019 at 7:58am an Apple tes...
2      On 01/10/19 at 2:52pm, an Aurora vehicle, whil...
3      A Cruise autonomous vehicle (“Cruise AV”), ope...
4      A Cruise autonomous vehicle (“Cruise AV”), ope...
                             ...                        
322    A Zoox vehicle in autonomous mode was stopped ...
323    A Zoox vehicle in manual mode was stopped at a...
324    A Zoox vehicle in autonomous mode was proceedi...
325    A Zoox vehicle in autonomous mode was stopped ...
326    A Zoox vehicle in autonomous mode was proceedi...
Name: AccDETAILS, Length: 327, dtype: object

In [ ]:
print(df_processed)

           AV_manufacturer  AVyear      AVmake     AVmodel    AVregloc  \
0           Aimotive , Inc  2010.0      Toyota       Prius          CA   
1                Apple Inc  2017.0       Lexus     RX 450h  California   
2    Aurora Innovation Inc  2017.0     Lincoln         MKZ  California   
3               Cruise LLC  2020.0  Cheverolet        Bolt          CA   
4               Cruise LLC  2020.0   Chevrolet        Bolt          CA   
..                     ...     ...         ...         ...         ...   
322               Zoox Inc  2016.0      Toyota  Highlander          CA   
323               Zoox Inc  2016.0      Toyota  Highlander          CA   
324               Zoox Inc  2016.0      Toyota  Highlander          CA   
325               Zoox Inc  2016.0      Toyota  Highlander          CA   
326               Zoox Inc  2016.0      Toyota  Highlander          CA   

     AccidentZIPcode AccidentSTATE AccidentCOUNTY AccidentCITY  \
0            94043.0            CA    Santa C

In [ ]:
df_filled = df_processed.fillna(' ')


In [ ]:
df_filled.to_csv('ProcessedDataset.csv')

In [ ]:
from sklearn.model_selection import train_test_split

train_size = 0.7

# Split the combined dataframe into training and validation sets
train_df, validation_df = train_test_split(df_filled, test_size=1-train_size, random_state=42)
display(len(train_df) )
display(len(validation_df) )


228

99

In [ ]:
#df_cleaned = df_pandas.dropna()

validation_df.isnull().sum()


AV_manufacturer                              0
AVyear                                       0
AVmake                                       0
AVmodel                                      0
AVregloc                                     0
                                            ..
AccOTHERfactorsDEFECTIVEWEHEQUIPNo-Status    0
AccOTHERfactorsUNINVOLVEDVEHICLE-Status      0
AccOTHERfactorsOTHER-Status                  0
AccOTHERfactorsNONEAPPARENT-Status           0
AccOTHERfactorsRUNAWAYVEHICLE-Status         0
Length: 194, dtype: int64

In [ ]:
train_df.to_csv('train_df.csv')
validation_df.to_csv('validation_df.csv')

In [ ]:
#train_df = pd.read_csv('train_df.csv')
#validation_df = pd.read_csv('validation_df.csv')

In [ ]:
train_df

,AV_manufacturer,AVyear,AVmake,AVmodel,AVregloc,AccidentZIPcode,AccidentSTATE,AccidentCOUNTY,AccidentCITY,AccidentADDRESS,...,AccOTHERfactorsSTOPGOTRAFFIC-Status,AccOTHERfactorsENTERINGORLEAVINGRAMP-Status,AccOTHERfactorsPREVIOUSCOLLISION-Status,AccOTHERfactorsUNFAMILIARWITHROAD-Status,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-Status,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-Status,AccOTHERfactorsUNINVOLVEDVEHICLE-Status,AccOTHERfactorsOTHER-Status,AccOTHERfactorsNONEAPPARENT-Status,AccOTHERfactorsRUNAWAYVEHICLE-Status
210,Waymo LLC,2017.0,Chrysler,Pacifica,CA,94109.0,CA,San Francisco,Francisco,Jones Street at Clay,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
81,Waymo LLC,2017.0,Chrysler,Pacifica,CA,94040.0,CA,Santa Clara,View,Escuela Avenue El Camino Real,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
194,Waymo LLC,2021.0,Jaguar,I-Pace,CA,94110.0,CA,San Francisco,Francisco,York Street Cesar Chavez,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
39,GM Cruise LLC,2019.0,Chevrolet,Bolt,CA,94103.0,CA,San Francisco,Francisco,Folsom St between 13th 12th,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
226,Waymo LLC,2021.0,Jaguar,I-Pace,CA,94110.0,CA,San Francisco,Francisco,Alabama Street at 24th,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Waymo LLC,2017.0,Chrysler,Pacifica,CA,94124.0,CA,San Francisco,Francisco,Oakdale Avenue at Industrial Street,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
71,"Pony.AI , Inc",2018.0,Lincoln,MKZ,CA,95035.0,CA,Alameda,,California Cir & Dixon Landing Rd,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
106,Cruise LLC,2020.0,Autonomous,9FM48,CA,94116.0,CA,San Francisco,Francisco,Taraval St 21st Ave,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,

In [ ]:
validation_df

,AV_manufacturer,AVyear,AVmake,AVmodel,AVregloc,AccidentZIPcode,AccidentSTATE,AccidentCOUNTY,AccidentCITY,AccidentADDRESS,...,AccOTHERfactorsSTOPGOTRAFFIC-Status,AccOTHERfactorsENTERINGORLEAVINGRAMP-Status,AccOTHERfactorsPREVIOUSCOLLISION-Status,AccOTHERfactorsUNFAMILIARWITHROAD-Status,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-Status,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-Status,AccOTHERfactorsUNINVOLVEDVEHICLE-Status,AccOTHERfactorsOTHER-Status,AccOTHERfactorsNONEAPPARENT-Status,AccOTHERfactorsRUNAWAYVEHICLE-Status
231,Waymo LLC,2021.0,Jaguar,I-Pace,MI,94123.0,CA,San Francisco,Francisco,Lombard Street Scott,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
110,Cruise LLC,2020.0,Chevrolet,Bolt,CA,94133.0,CA,San Francisco,Francisco,1170 Columbus Av,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
250,Cruise LLC,2020.0,Cruise,AV,CA,94103.0,CA,San Francisco,Francisco,3001 16th St,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
9,Cruise LLC,2020.0,Chevrolet,Bolt,CA,94110.0,CA,San Francisco,Francisco,24th St Mission,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
93,Waymo LLC,2017.0,Chrysler,Pacifica,CA,94043.0,CA,Santa Clara,View,Whisman Station Drive Central Expressway,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,Waymo LLC,2021.0,Jaguar,I-Pace,CA,94115.0,CA,San Francisco,Francisco,Pine Street at Fillmore,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
253,Cruise LLC,2020.0,Cruise,AV,CA,94109.0,CA,San Francisco,Francisco,1798 Sacramento St,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFECTIVEWEHEQUIPNo-No,AccOTHERfactorsUNINVOLVEDVEHICLE-No,AccOTHERfactorsOTHER-No,AccOTHERfactorsNONEAPPARENT-No,AccOTHERfactorsRUNAWAYVEHICLE-No
56,GM Cruise LLC,2020.0,Chevrolet,Bolt,CA,94114.0,CA,San Francisco,Francisco,14th St Noe,...,AccOTHERfactorsSTOPGOTRAFFIC-No,AccOTHERfactorsENTERINGORLEAVINGRAMP-No,AccOTHERfactorsPREVIOUSCOLLISION-No,AccOTHERfactorsUNFAMILIARWITHROAD-No,AccOTHERfactorsDEFECTIVEWEHEQUIPYes-No,AccOTHERfactorsDEFE

In [ ]:
train_df['FullText'] =  train_df.apply(lambda row: ', '.join([f'{col}:{value}' for col, value in zip(train_df.columns, row) if col != 'AccDETAILS']), axis=1)

validation_df['FullText'] =  validation_df.apply(lambda row: ', '.join([f'{col}:{value}' for col, value in zip(validation_df.columns, row) if col != 'AccDETAILS']), axis=1)



In [ ]:

# Print the formatted data
validation_df['FullText'].iloc[88]


'AV_manufacturer:GM Cruise LLC, AVyear:2020.0, AVmake:Chevrolet, AVmodel:Bolt, AVregloc:CA, AccidentZIPcode:94102.0, AccidentSTATE:CA, AccidentCOUNTY:San Francisco, AccidentCITY:Francisco, AccidentADDRESS:Haight St Gough, AccidentDATE:09/05/2019, AccidentTIME:05:33, AVnoOFVehicles:2.0, V2year: , V2model:Vespa, V2hitOTHERSspecify: , V2noOFVehicles:2.0, AccTYPE(Mode):Conventional, AccidenttimeOFdayAM-Status:AccidenttimeOFdayAM-No, AccidenttimeOFdayPM-Status:AccidenttimeOFdayPM-Yes, AVmotionMOVING-Status:AVmotionMOVING-Yes, AVmotionSTOPPED-Status:AVmotionSTOPPED-No, AVhitPEDESTRAIN-Status:AVhitPEDESTRAIN-No, AVhitBICYCLIST-Status:AVhitBICYCLIST-No, AVhitOTHERS-Status:AVhitOTHERS-No, AVhitOTHERSspecify-Status:AVhitOTHERSspecify-No, AVdamagelevelUNKNOWN-Status:AVdamagelevelUNKNOWN-No, AVdamagelevelNONE-Status:AVdamagelevelNONE-No, AVdamagelevelMINOR-Status:AVdamagelevelMINOR-Yes, AVdamagelevelMODERATE-Status:AVdamagelevelMODERATE-No, AVdamagelevelMAJOR-Status:AVdamagelevelMAJOR-No, AVdamage

In [ ]:
validation_df['AccDETAILS'].iloc[88]


'A Cruise autonomous vehicle (“Cruise AV”) being driven in conventional mode was involved in a collision after completing a right turn from southwest bound Market Street onto westbound Haight Street at the intersection with Gough Street. As the driver of the Cruise AV proceeded on Haight through Gough, a motor scooterist, proceeding southbound on Gough through the intersection with Haight, made contact with the right rear passenger door of the Cruise AV, denting the door. There were no injuries reported at the scene by either party. Police were called but not dispatched to the scene. Days after the collision, the motor scooterist reported injuries.'

In [ ]:


#{"role": "system", "content": 'You are a seasoned legal professional and a researcher, graduated with multiple distinctions specializing in lawsuit cases summarization. You are involved in a NLP research to make a summarizer. Given the long summary extracted from legal documents, your job is to provide a shorter summary having single paragraph around 130 words for the case all according to the criteria given by your supervisor.'},

messages_list = []
for index, row in train_df.iterrows():
    #cleaned_text = re.sub(r'[^a-zA-Z\s]', '', str(row['summary/long']))
    #print(cleaned_text)

    messages_list.append([
        {"role": "system", "content": 'You are the best Autonomous Vehicle Crash Expert Analyst who can prepare crash descriptive reports from the given data and different factors.'},
        {"role": "user", "content": str(row['FullText'])},  # Convert to string if necessary
        {"role": "assistant", "content": str(row['AccDETAILS'])}  # Convert to string if necessary
    ])
    #x=num_tokens_from_string(cleaned_text, "cl100k_base")
    #y=num_tokens_from_string(str(row['summary/short']),"cl100k_base")
    #print(int(x))
    #print(int(y))


# Write each dictionary as a separate line in a JSON file (JSONL format)
with open('output_train.jsonl', 'w') as json_file:
    for messages in messages_list:
        json_line = json.dumps({"messages": messages})
        json_file.write(json_line + '\n')







messages_list = []
for index, row in validation_df.iterrows():
    #cleaned_text = re.sub(r'[^a-zA-Z\s]', '', str(row['summary/long']))
    #print(cleaned_text)

    messages_list.append([
        {"role": "system", "content": 'You are a best legal expert who can prepare a summary of the whole case when given.'},
        {"role": "user", "content": str(row['FullText'])},  # Convert to string if necessary
        {"role": "assistant", "content": str(row['AccDETAILS'])}  # Convert to string if necessary
    ])
    #x=num_tokens_from_string(cleaned_text, "cl100k_base")
    #y=num_tokens_from_string(str(row['summary/short']),"cl100k_base")
    #print(int(x))
    #print(int(y))


# Write each dictionary as a separate line in a JSON file (JSONL format)
with open('output_valid.jsonl', 'w') as json_file:
    for messages in messages_list:
        json_line = json.dumps({"messages": messages})
        json_file.write(json_line + '\n')

In [ ]:
with open('output_train.jsonl', 'r') as json_file:
  print(json_file.readlines())


['{"messages": [{"role": "system", "content": "You are the best Autonomous Vehicle Crash Expert Analyst who can prepare crash descriptive reports from the given data and different factors."}, {"role": "user", "content": "AV_manufacturer:Waymo LLC, AVyear:2017.0, AVmake:Chrysler, AVmodel:Pacifica, AVregloc:CA, AccidentZIPcode:94109.0, AccidentSTATE:CA, AccidentCOUNTY:San Francisco, AccidentCITY:Francisco, AccidentADDRESS:Jones Street at Clay, AccidentDATE:04/04/2021, AccidentTIME:08:50, AVnoOFVehicles:2.0, V2year:2016.0, V2model:Honda CR-V, V2hitOTHERSspecify: , V2noOFVehicles:2.0, AccTYPE(Mode):Conventional, AccidenttimeOFdayAM-Status:AccidenttimeOFdayAM-Yes, AccidenttimeOFdayPM-Status:AccidenttimeOFdayPM-No, AVmotionMOVING-Status:AVmotionMOVING-Yes, AVmotionSTOPPED-Status:AVmotionSTOPPED-No, AVhitPEDESTRAIN-Status:AVhitPEDESTRAIN-No, AVhitBICYCLIST-Status:AVhitBICYCLIST-No, AVhitOTHERS-Status:AVhitOTHERS-No, AVhitOTHERSspecify-Status:AVhitOTHERSspecify-No, AVdamagelevelUNKNOWN-Status:

In [ ]:
with open('output_valid.jsonl', 'r') as json_file:
  print(json_file.readlines())

['{"messages": [{"role": "system", "content": "You are a best legal expert who can prepare a summary of the whole case when given."}, {"role": "user", "content": "AV_manufacturer:Aimotive , Inc, AVyear:2010.0, AVmake:Toyota, AVmodel:Prius, AVregloc:CA, AccidentZIPcode:94043.0, AccidentSTATE:CA, AccidentCOUNTY:Santa Clara, AccidentCITY:View, AccidentADDRESS:SB , 101 US Freeway Old Middlefield Rd x Shorline, AccidentDATE:09/16/2019, AccidentTIME:10:00, AVnoOFVehicles:2.0, V2year:2007.0, V2model:Honda CRV, V2hitOTHERSspecify: , V2noOFVehicles:2.0, AccTYPE(Mode):Conventional, AccidenttimeOFdayAM-Status:AccidenttimeOFdayAM-Yes, AccidenttimeOFdayPM-Status:AccidenttimeOFdayPM-No, AVmotionMOVING-Status:AVmotionMOVING-No, AVmotionSTOPPED-Status:AVmotionSTOPPED-Yes, AVhitPEDESTRAIN-Status:AVhitPEDESTRAIN-No, AVhitBICYCLIST-Status:AVhitBICYCLIST-No, AVhitOTHERS-Status:AVhitOTHERS-No, AVhitOTHERSspecify-Status:AVhitOTHERSspecify-No, AVdamagelevelUNKNOWN-Status:AVdamagelevelUNKNOWN-No, AVdamageleve

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='sk-iii')


filename= 'output_train.jsonl'
x= client.files.create(
  file=open(filename, "rb"),
  purpose="fine-tune"
)
filename= 'output_valid.jsonl'

y= client.files.create(
  file=open(filename, "rb"),
  purpose="fine-tune"
)

print(x)
print(y)

FileObject(id='file-2nlQ85v717ALlnB1vabgtns1', bytes=159248, created_at=1708897413, filename='output_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FileObject(id='file-I58VdKO5y9wNNJ69mPOOq1gw', bytes=86163, created_at=1708897414, filename='output_valid.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [ ]:
from openai import OpenAI
client = OpenAI(api_key='sk-iiii')
# change filenames from above and paste it below
client.fine_tuning.jobs.create(
  training_file='file-2nlQ85v717ALlnB1vabgtns1',
  validation_file='file-I58VdKO5y9wNNJ69mPOOq1gw',
  model="gpt-3.5-turbo-1106"
)


FineTuningJob(id='ftjob-EQhhSv1XPagfXw8ajvPM6kny', created_at=1708897424, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-mwA0QIWpHg7raxJ186XN15FZ', result_files=[], status='validating_files', trained_tokens=None, training_file='file-2nlQ85v717ALlnB1vabgtns1', validation_file='file-I58VdKO5y9wNNJ69mPOOq1gw')

In [ ]:
client.fine_tuning.jobs.retrieve("ftjob-9U9QAEtsKtk22B1LhdVR3cAK")


FineTuningJob(id='ftjob-9U9QAEtsKtk22B1LhdVR3cAK', created_at=1708897333, error=Error(code='invalid_n_examples', message='Training file has 6 example(s), but must have at least 10 examples', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-mwA0QIWpHg7raxJ186XN15FZ', result_files=[], status='failed', trained_tokens=None, training_file='file-KhWJkOWsrZZIqdvSbPY5MXUo', validation_file='file-MGBYzdAjhhGyEKULg11FMdcm')

In [ ]:
validation_df['AccDETAILS']

231    On September 22, 2021 at 4:29 AM PDT a Waymo A...
110    A Cruise autonomous vehicle (“Cruise AV”), ope...
250    A Cruise autonomous vehicle ("Cruise AV"), ope...
9      A Cruise autonomous vehicle (“Cruise AV”), ope...
93     A Waymo Autonomous Vehicle (“Waymo AV”) was st...
                             ...                        
278    On February 6, 2022 at 1:53 PM PST a Waymo Aut...
253    A Cruise autonomous vehicle ("Cruise AV"), ope...
56     A Cruise autonomous vehicle (“Cruise AV”), ope...
222    On July 12, 2021 at 3:10 PM PDT, a Waymo Auton...
137    A Zoox vehicle was parked with hazard lights b...
Name: AccDETAILS, Length: 99, dtype: object

In [ ]:
print(validation_df['FullText'].loc[278])
print(validation_df['AccDETAILS'].loc[93])

AV_manufacturer:Waymo LLC, AVyear:2021.0, AVmake:Jaguar, AVmodel:I-Pace, AVregloc:CA, AccidentZIPcode:94115.0, AccidentSTATE:CA, AccidentCOUNTY:San Francisco, AccidentCITY:Francisco, AccidentADDRESS:Pine Street at Fillmore, AccidentDATE:02/06/2022, AccidentTIME:01:53, AVnoOFVehicles:3.0, V2year:2016.0, V2model:Ford Fusion, V2hitOTHERSspecify: , V2noOFVehicles:3.0, AccTYPE(Mode):Conventional, AccidenttimeOFdayAM-Status:AccidenttimeOFdayAM-No, AccidenttimeOFdayPM-Status:AccidenttimeOFdayPM-Yes, AVmotionMOVING-Status:AVmotionMOVING-No, AVmotionSTOPPED-Status:AVmotionSTOPPED-Yes, AVhitPEDESTRAIN-Status:AVhitPEDESTRAIN-No, AVhitBICYCLIST-Status:AVhitBICYCLIST-No, AVhitOTHERS-Status:AVhitOTHERS-No, AVhitOTHERSspecify-Status:AVhitOTHERSspecify-No, AVdamagelevelUNKNOWN-Status:AVdamagelevelUNKNOWN-No, AVdamagelevelNONE-Status:AVdamagelevelNONE-No, AVdamagelevelMINOR-Status:AVdamagelevelMINOR-No, AVdamagelevelMODERATE-Status:AVdamagelevelMODERATE-Yes, AVdamagelevelMAJOR-Status:AVdamagelevelMAJOR

In [ ]:
# This code is for v1 of the openai package: pypi.org/project/openai
from openai import OpenAI
indexnumber= 222
#
client = OpenAI(api_key='sk-ii')

print('Original Report:', validation_df['AccDETAILS'].loc[indexnumber])
print('***************')
#very old model 8Nu5IzXV  0.09 loss training
# newest model 8TdwOEyz  0.9
#0613:personal::8TenGxpP  0.4 , 0.7 precision,
#8TfHwWmI 0613 , 50 samples,
#8U4FS4WK 1106 , 70-30 rule, 100% accuracy bht fit
response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-1106:personal::8wGfkLcE",
  messages=[
         {
      "role": "system",
      "content": "You are the best Autonomous Vehicle Crash Expert Analyst who can prepare crash descriptive reports from the given data and different factors.."
    },
    {
      "role": "user",
      "content": validation_df['FullText'].loc[indexnumber]
,
    }
  ],
  temperature=0,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0

)
#0.3 temp pe sb sse ziada ache results ate.

content = response.choices[0].message.content

# Print or use the content as needed
print('AI Generated Report:')
print(content)
from nltk import ngrams
from nltk.metrics import edit_distance

def calculate_rouge_n(reference, hypothesis, n):
    reference_ngrams = set(ngrams(reference, n))
    hypothesis_ngrams = set(ngrams(hypothesis, n))

    intersection = len(reference_ngrams.intersection(hypothesis_ngrams))
    recall = intersection / len(reference_ngrams)
    precision = intersection / len(hypothesis_ngrams) if len(hypothesis_ngrams) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {'precision': precision, 'recall': recall, 'f1_score': f1_score}

def calculate_rouge_l(reference, hypothesis):
    reference_lcs = set(reference).intersection(hypothesis)
    recall = len(reference_lcs) / len(reference)
    precision = len(reference_lcs) / len(hypothesis) if len(hypothesis) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {'precision': precision, 'recall': recall, 'f1_score': f1_score}

def preprocess_text(text):
    # You may want to add additional text preprocessing steps here
    return text.lower().split()

# Example usage:
reference_summary = content
hypothesis_summary = validation_df['AccDETAILS'].loc[indexnumber]




reference_tokens = preprocess_text(reference_summary)
hypothesis_tokens = preprocess_text(hypothesis_summary)

rouge_1_scores = calculate_rouge_n(reference_tokens, hypothesis_tokens, 1)
rouge_2_scores = calculate_rouge_n(reference_tokens, hypothesis_tokens, 2)
rouge_l_scores = calculate_rouge_l(reference_tokens, hypothesis_tokens)

print("ROUGE-1:", rouge_1_scores)
print("ROUGE-2:", rouge_2_scores)
print("ROUGE-L:", rouge_l_scores)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

def calculate_cosine_similarity(text1, text2):
    vectorizer = CountVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()

    # Reshape the vectors for cosine_similarity function
    vectors = vectors.reshape(2, -1)

    # Calculate cosine similarity
    similarity = cosine_similarity(vectors[0].reshape(1, -1), vectors[1].reshape(1, -1))[0, 0]

    return similarity



cosine_similarity_score = calculate_cosine_similarity(reference_summary, hypothesis_summary)

print("Cosine Similarity Score:", cosine_similarity_score)



def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    similarity = intersection / union if union != 0 else 0.0
    return similarity

# Example usage:
text1 = reference_summary
text2 = hypothesis_summary
# Tokenize the texts into sets of words
set1 = set(text1.split())
set2 = set(text2.split())

# Calculate Jaccard similarity
similarity_score = jaccard_similarity(set1, set2)

print(f"Jaccard Similarity: {similarity_score}")

Original Report: On July 12, 2021 at 3:10 PM PDT, a Waymo Autonomous Vehicle (“Waymo AV”) was in a collision involving a van at Broadway and Gough Street in San Francisco. The Waymo AV, while in autonomous mode, was waiting to make a left turn and yielding to oncoming traffic, when the test driver (in the driver’s seating position) transitioned to manual mode and proceeded with a left turn manually. During the left turn, an approaching van struck the right side of the Waymo AV. At the time of the impact, the Waymo AV was in manual mode.
***************
AI Generated Report:
A Waymo autonomous vehicle (“Waymo”) operating in autonomous mode was traveling eastbound on Broadway Street at the intersection with Gough Street when the driver of the other vehicle, who was in the left turn lane, made a wide left turn and struck the right rear corner of the Waymo vehicle. The contact caused damage to the Waymo vehicle’s right rear bumper, radar, and tail light. There were no injuries and police we

In [ ]:
# This code is for v1 of the openai package: pypi.org/project/openai
print('Original report:', df_testing['summary/tiny'].loc[406])
print('***************')

response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-1106:personal::8U4FS4WK",
  messages=[
         {
      "role": "system",
      "content": "You are a best legal expert who can prepare a summary of the whole case when given."
    },
    {
      "role": "user",
      "content": df_testing['summary/short'].loc[406],
    }
  ],
  temperature=0,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0

)
#0.3 temp pe sb sse ziada ache results ate.

content = response.choices[0].message.content

# Print or use the content as needed
print('AI Generated tiny Summary:')
print(content)
from nltk import ngrams
from nltk.metrics import edit_distance

def calculate_rouge_n(reference, hypothesis, n):
    reference_ngrams = set(ngrams(reference, n))
    hypothesis_ngrams = set(ngrams(hypothesis, n))

    intersection = len(reference_ngrams.intersection(hypothesis_ngrams))
    recall = intersection / len(reference_ngrams)
    precision = intersection / len(hypothesis_ngrams) if len(hypothesis_ngrams) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {'precision': precision, 'recall': recall, 'f1_score': f1_score}

def calculate_rouge_l(reference, hypothesis):
    reference_lcs = set(reference).intersection(hypothesis)
    recall = len(reference_lcs) / len(reference)
    precision = len(reference_lcs) / len(hypothesis) if len(hypothesis) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {'precision': precision, 'recall': recall, 'f1_score': f1_score}

def preprocess_text(text):
    # You may want to add additional text preprocessing steps here
    return text.lower().split()

# Example usage:
reference_summary = content
hypothesis_summary = df_testing['summary/tiny'].loc[406]




reference_tokens = preprocess_text(reference_summary)
hypothesis_tokens = preprocess_text(hypothesis_summary)

rouge_1_scores = calculate_rouge_n(reference_tokens, hypothesis_tokens, 1)
rouge_2_scores = calculate_rouge_n(reference_tokens, hypothesis_tokens, 2)
rouge_l_scores = calculate_rouge_l(reference_tokens, hypothesis_tokens)

print("ROUGE-1:", rouge_1_scores)
print("ROUGE-2:", rouge_2_scores)
print("ROUGE-L:", rouge_l_scores)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

def calculate_cosine_similarity(text1, text2):
    vectorizer = CountVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()

    # Reshape the vectors for cosine_similarity function
    vectors = vectors.reshape(2, -1)

    # Calculate cosine similarity
    similarity = cosine_similarity(vectors[0].reshape(1, -1), vectors[1].reshape(1, -1))[0, 0]

    return similarity



cosine_similarity_score = calculate_cosine_similarity(reference_summary, hypothesis_summary)

print("Cosine Similarity Score:", cosine_similarity_score)



def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    similarity = intersection / union if union != 0 else 0.0
    return similarity

# Example usage:
text1 = reference_summary
text2 = hypothesis_summary
# Tokenize the texts into sets of words
set1 = set(text1.split())
set2 = set(text2.split())

# Calculate Jaccard similarity
similarity_score = jaccard_similarity(set1, set2)

print(f"Jaccard Similarity: {similarity_score}")

Original Tiny Summary: ICE Agent's claims that deferred action violates federal law dismissed (N.D. Tex.)
***************
AI Generated tiny Summary:
ICE Agents and Mississippi challenge deferred action directive. (S.D. Tex.)
ROUGE-1: {'precision': 0.3333333333333333, 'recall': 0.4, 'f1_score': 0.3636363636363636}
ROUGE-2: {'precision': 0.09090909090909091, 'recall': 0.1111111111111111, 'f1_score': 0.09999999999999999}
ROUGE-L: {'precision': 0.3333333333333333, 'recall': 0.4, 'f1_score': 0.3636363636363636}
Cosine Similarity Score: 0.40201512610368484
Jaccard Similarity: 0.2222222222222222


In [ ]:
!pip install openai
from openai import OpenAI
client = OpenAI(api_key='sk-hhh')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
